In [1]:
import traffic
from traffic.data import opensky
from traffic.core import Traffic
from rich.pretty import pprint
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import h5py
import os

In [2]:
def download_month(month: int, year: int, start_day = 1, cached=True):

    if month < 10:
        str_month = "0" + str(month)
    else:
        str_month = str(month)

    filename = "Frankfurt_LH_" + str(year)[2:] + str_month + ".h5"
    print(filename)
    next_month = month + 1 if month < 12 else  1
    str_next_month = str(next_month) if next_month > 9 else "0" + str(next_month)

    start_day_str = "0" + str(start_day) if start_day < 10 else str(start_day)
    datetime_str_start = str(year) + "-"+ str_month + "-" + start_day_str +  " 00:00:00"
    datetime_str_end = str(year) + "-" + str_next_month + "-01 00:00:00" if next_month != 1 else str(year+1) + "-" + str_next_month + "-01 00:00:00"
    datetime_current = datetime.strptime(datetime_str_start, '%Y-%m-%d %H:%M:%S')
    datetime_end = datetime.strptime(datetime_str_end, '%Y-%m-%d %H:%M:%S')

    while datetime_current < datetime_end:

        print(datetime_current)
        day = datetime_current.day
        if day < 10:
            day_str = "0" + str(day)
        else:
            day_str = str(day)

        h5_key = "LH_22" + str_month + day_str
        print(h5_key)
        str_current_day = str(datetime_current)
        str_next_day = str(datetime_current + timedelta(days = 1))
        print("getting data between "+str_current_day+ " and "+ str_next_day)

        trajectories = opensky.history(str_current_day,
                            stop= str_next_day,
                            arrival_airport ="EDDF",
                            cached=cached)
        try:
            callsigns_lh = trajectories.data.callsign.loc[trajectories.data.callsign.str.contains("DLH").astype(bool)]

            callsigns_lh = callsigns_lh.unique()
            callsigns_lh = callsigns_lh[~pd.isnull(callsigns_lh)]
            trajectories_lh = trajectories[callsigns_lh]
            print("add",h5_key,"to file")

            trajectories_lh.to_hdf(filename, key=h5_key, format = 'table')

        except AttributeError as e:
            print(e)

        datetime_current += timedelta(days = 1)


In [ ]:
download_month(3, 2023, 12, cached = True)

Output()

Frankfurt_LH_2303.h5
2023-03-12 00:00:00
LH_220312
getting data between 2023-03-12 00:00:00 and 2023-03-13 00:00:00


C:\Users\dario\anaconda3\envs\Lufthansa-Arrival-Time-Prediction\lib\site-packages\traffic\data\adsb\opensky_impala.
py:196: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(s, dtype={"icao24": str, "callsign": str})

add LH_220312 to file


Output()

2023-03-13 00:00:00
LH_220313
getting data between 2023-03-13 00:00:00 and 2023-03-14 00:00:00


C:\Users\dario\anaconda3\envs\Lufthansa-Arrival-Time-Prediction\lib\site-packages\traffic\data\adsb\opensky_impala.
py:196: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(s, dtype={"icao24": str, "callsign": str})

add LH_220313 to file


Output()

2023-03-14 00:00:00
LH_220314
getting data between 2023-03-14 00:00:00 and 2023-03-15 00:00:00


In [3]:
with h5py.File("Frankfurt_LH_2303.h5", 'r') as f:
    keys = print(list(f.keys()))


['LH_220301', 'LH_220302', 'LH_220303', 'LH_220304', 'LH_220305', 'LH_220306', 'LH_220307', 'LH_220308', 'LH_220309', 'LH_220310', 'LH_220311', 'LH_220312', 'LH_220313', 'LH_220314', 'LH_220315', 'LH_220316', 'LH_220317', 'LH_220318', 'LH_220319', 'LH_220320']
